# Neural Cognitive Diagnosis Model (NCDM)
This notebook will show you how to train and use the NCDM. First, we will show how to get the data (here we use a0910 as the dataset). Then we will show how to train a NCDM and perform the parameters persistence. At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [NCDM.py](NCDM.py)

# Data Preparation
Before we process the data, we need to first acquire the dataset which is shown in this [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [1]:
# Load the data from files
import pandas as pd

train_data = pd.read_csv("../../data/a0910/train.csv")
valid_data = pd.read_csv("../../data/a0910/valid.csv")
test_data = pd.read_csv("../../data/a0910/test.csv")
df_item = pd.read_csv("../../data/a0910/item.csv")
item2knowledge = {}
knowledge_set = set()
for i, s in df_item.iterrows():
    item_id, knowledge_codes = s['item_id'], list(set(eval(s['knowledge_code'])))
    item2knowledge[item_id] = knowledge_codes
    knowledge_set.update(knowledge_codes)

train_data.head(5)

,user_id,item_id,score
0,1615,12977,1
1,782,13124,0
2,1084,16475,0
3,593,8690,0
4,127,14225,1


In [2]:
len(train_data), len(valid_data), len(test_data)

(186049, 25606, 55760)

In [3]:
# Get basic data info for model initialization
import numpy as np
user_n = np.max(train_data['user_id'])
item_n = np.max([np.max(train_data['item_id']), np.max(valid_data['item_id']), np.max(test_data['item_id'])])
knowledge_n = np.max(list(knowledge_set))

user_n, item_n, knowledge_n

(4128, 17746, 123)

In [4]:
# Transform data to torch Dataloader (i.e., batchify)
# batch_size is set to 32

import torch
from torch.utils.data import TensorDataset, DataLoader

batch_size = 32
def transform(user, item, item2knowledge, score, batch_size):
    knowledge_emb = torch.zeros((len(item), knowledge_n))
    for idx in range(len(item)):
        knowledge_emb[idx][np.array(item2knowledge[item[idx]]) - 1] = 1.0

    data_set = TensorDataset(
        torch.tensor(user, dtype=torch.int64) - 1,  # (1, user_n) to (0, user_n-1)
        torch.tensor(item, dtype=torch.int64) - 1,  # (1, item_n) to (0, item_n-1)
        knowledge_emb,
        torch.tensor(score, dtype=torch.float32)
    )
    return DataLoader(data_set, batch_size=batch_size, shuffle=True)


train_set, valid_set, test_set = [
    transform(data["user_id"], data["item_id"], item2knowledge, data["score"], batch_size)
    for data in [train_data, valid_data, test_data]
]

train_set, valid_set, test_set

(<torch.utils.data.dataloader.DataLoader at 0x2b8f1ecee20>,
 <torch.utils.data.dataloader.DataLoader at 0x2b8f1eba3a0>)

# Training and Persistence

In [5]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [6]:
from EduCDM import NCDM

cdm = NCDM(knowledge_n, item_n, user_n)
cdm.train(train_set, valid_set, epoch=3, device="cuda")
cdm.save("ncdm.snapshot")

Evaluating:   5%|███▍                                                                | 41/801 [00:00<00:02, 356.55it/s]

[Epoch 0] average loss: 0.650993


Epoch 1:   0%|                                                                       | 10/5815 [00:00<01:06, 86.67it/s]

[Epoch 0] auc: 0.712843, accuracy: 0.661056


Evaluating:   5%|███▍                                                                | 40/801 [00:00<00:01, 397.00it/s]

...

[Epoch 1] auc: 0.741101, accuracy: 0.722018


Evaluating:   5%|███▍                                                                | 41/801 [00:00<00:01, 394.46it/s]

[Epoch 2] average loss: 0.466515


Evaluating: 100%|███████████████████████████████████████████████████████████████████| 801/801 [00:02<00:00, 390.05it/s]
INFO:root:save parameters to ncdm.snapshot


[Epoch 2] auc: 0.749467, accuracy: 0.728032


# Loading and Testing

In [7]:
cdm.load("ncdm.snapshot")
auc, accuracy = cdm.eval(test_set)
print("auc: %.6f, accuracy: %.6f" % (auc, accuracy))

INFO:root:load parameters from ncdm.snapshot
Evaluating: 100%|█████████████████████████████████████████████████████████████████| 1743/1743 [00:02<00:00, 763.27it/s]

auc: 0.751848, accuracy: 0.730273
